## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-17-13-00-00 +0000,wsj,That Single-Use Coffee Cup Could Be on the Way...,https://www.wsj.com/business/hospitality/eu-co...
1,2025-09-17-12-58-02 +0000,nypost,Spirit Airlines pilots scolded by air traffic ...,https://nypost.com/2025/09/17/us-news/spirit-a...
2,2025-09-17-12-56-51 +0000,nyt,Israel Pounds Gaza City as Fears Mount for Tho...,https://www.nytimes.com/2025/09/17/world/middl...
3,2025-09-17-12-55-46 +0000,nyt,100 Liberal Philanthropies Prepare to Push Bac...,https://www.nytimes.com/2025/09/17/us/politics...
4,2025-09-17-12-54-30 +0000,nyt,Live Updates: U.K. Gives Trump a Royal Welcome...,https://www.nytimes.com/live/2025/09/17/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,67
107,kirk,38
144,charlie,26
22,gaza,24
23,city,22


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
155,2025-09-17-03-28-00 +0000,wsj,In her first public comments since Trump fired...,https://www.wsj.com/economy/jobs/fired-bls-chi...,165
58,2025-09-17-10-47-31 +0000,nypost,Trump claims Biden admin weaponized DOJ to tar...,https://nypost.com/2025/09/17/us-news/trump-cl...,162
106,2025-09-17-09-03-58 +0000,nyt,"As Trump Arrives, Europe’s Right Claims Charli...",https://www.nytimes.com/2025/09/17/world/europ...,161
71,2025-09-17-10-00-00 +0000,latimes,A Pride flag at half-staff to honor Charlie Ki...,https://www.latimes.com/california/story/2025-...,146
365,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...,130


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
155,165,2025-09-17-03-28-00 +0000,wsj,In her first public comments since Trump fired...,https://www.wsj.com/economy/jobs/fired-bls-chi...
365,121,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...
35,108,2025-09-17-11-44-17 +0000,cbc,Israel opens new temporary route out of Gaza C...,https://www.cbc.ca/news/world/gaza-city-israel...
250,106,2025-09-16-20-14-00 +0000,wsj,Democrats are introducing a bill in the Senate...,https://www.wsj.com/economy/central-banking/se...
1,55,2025-09-17-12-58-02 +0000,nypost,Spirit Airlines pilots scolded by air traffic ...,https://nypost.com/2025/09/17/us-news/spirit-a...
125,53,2025-09-17-07-09-00 +0000,nypost,Ben & Jerry’s co-founder Jerry Greenfield resi...,https://nypost.com/2025/09/17/business/ben-amp...
207,49,2025-09-16-22-28-55 +0000,nypost,Hochul argues Zohran Mamdani would be best can...,https://nypost.com/2025/09/16/us-news/hochul-a...
39,44,2025-09-17-11-31-00 +0000,wsj,Heightened fears of politically motivated atta...,https://www.wsj.com/politics/policy/rattled-la...
368,43,2025-09-16-14-41-00 +0000,cbc,New York judge tosses terrorism charges agains...,https://www.cbc.ca/news/judge-dismisses-terror...
17,41,2025-09-17-12-29-00 +0000,wsj,"Republican Brad Raffensperger, the election ch...",https://www.wsj.com/politics/elections/georgia...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
